# K₇ Heat Kernel Spectral Estimator

## Méthode: Trace du Heat Kernel

**Principe**: Extraire λ₁ depuis la décroissance exponentielle de Tr(e^{-tL})

```
Tr(e^{-tL}) = Σₙ e^{-λₙt} = N·e^{-λ₀t} + (N-1)·e^{-λ₁t} + ...
           ≈ N + (N-1)·e^{-λ₁t}  pour t petit (λ₀ ≈ 0)
```

**Avantage**: La trace est une quantité INTRINSÈQUE — indépendante de la base.

**Extraction de λ₁**:
```
log(Tr(e^{-tL}) - N) ≈ log(N-1) - λ₁·t
→ Pente = -λ₁
```

In [ ]:
import numpy as np
import json
from datetime import datetime
from scipy.linalg import expm

try:
    import cupy as cp
    GPU = True
    print("✓ GPU (CuPy)")
except:
    GPU = False
    print("○ CPU mode")

print(f"Started: {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
# GIFT Constants
H_STAR = 99
DIM_G2 = 14
DET_G = 65/32
RATIO = H_STAR / 84

def sample_TCS_K7(N, seed):
    """Sample K₇ via TCS: S¹ × S³ × S³"""
    rng = np.random.default_rng(seed)
    theta = rng.uniform(0, 2*np.pi, N)
    def sample_S3(n):
        x = rng.standard_normal((n, 4))
        return x / np.linalg.norm(x, axis=1, keepdims=True)
    return theta, sample_S3(N), sample_S3(N)

In [ ]:
def compute_distance_matrix(theta, q1, q2, chunk=1500):
    """TCS geodesic distances."""
    N = len(theta)
    alpha = DET_G / (RATIO ** 3)
    D = np.zeros((N, N), dtype=np.float32)
    
    for i in range(0, N, chunk):
        ie = min(i + chunk, N)
        for j in range(0, N, chunk):
            je = min(j + chunk, N)
            
            diff = np.abs(theta[i:ie, None] - theta[None, j:je])
            d_S1 = np.minimum(diff, 2*np.pi - diff)
            
            d_S3_1 = np.zeros((ie-i, je-j), dtype=np.float32)
            d_S3_2 = np.zeros((ie-i, je-j), dtype=np.float32)
            for ii, (qi1, qi2) in enumerate(zip(q1[i:ie], q2[i:ie])):
                dot1 = np.clip(np.abs(np.sum(qi1 * q1[j:je], axis=1)), -1, 1)
                d_S3_1[ii] = 2 * np.arccos(dot1)
                dot2 = np.clip(np.abs(np.sum(qi2 * q2[j:je], axis=1)), -1, 1)
                d_S3_2[ii] = 2 * np.arccos(dot2)
            
            D[i:ie, j:je] = np.sqrt(alpha * d_S1**2 + d_S3_1**2 + RATIO**2 * d_S3_2**2)
    
    return D

In [ ]:
def compute_normalized_laplacian(D, sigma):
    """Standard normalized Laplacian."""
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    d = W.sum(axis=1)
    d_inv_sqrt = 1.0 / np.sqrt(d + 1e-10)
    
    # L = I - D^{-1/2} W D^{-1/2}
    L = np.eye(W.shape[0]) - (d_inv_sqrt[:, None] * W * d_inv_sqrt[None, :])
    return L

## Heat Kernel Trace Method

Pour une matrice L avec eigenvalues 0 = λ₀ < λ₁ ≤ λ₂ ≤ ...:

```
Tr(e^{-tL}) = Σₙ e^{-λₙt}
```

À t modéré, les premiers termes dominent:
```
Tr(e^{-tL}) ≈ 1 + (multiplicité de λ₁) × e^{-λ₁t} + ...
```

En pratique, on fit:
```
log(Tr(e^{-tL}) - 1) vs t  →  pente = -λ₁
```

In [ ]:
def heat_kernel_trace(L, t):
    """Compute Tr(e^{-tL}) via matrix exponential."""
    # For small matrices, direct expm is fine
    # For large matrices, use eigendecomposition or Krylov methods
    heat = expm(-t * L)
    return np.trace(heat)

def extract_lambda1_from_heat_trace(L, t_values):
    """Extract λ₁ from heat kernel trace decay.
    
    Method: fit log(Tr(e^{-tL}) - 1) vs t
    The slope gives -λ₁
    """
    N = L.shape[0]
    traces = []
    
    for t in t_values:
        tr = heat_kernel_trace(L, t)
        traces.append(tr)
    
    traces = np.array(traces)
    
    # Tr(e^{-tL}) - 1 ≈ (N-1)e^{-λ₁t} for normalized Laplacian
    # Actually for normalized Laplacian, Tr = N at t=0, and decays
    # At t=0: Tr(I) = N, but Tr(e^{-tL}) → multiplicity of λ=0 as t→∞
    
    # For normalized Laplacian: λ₀ = 0 with mult 1
    # So Tr(e^{-tL}) = 1 + Σ_{n≥1} e^{-λₙt}
    # At large t: Tr → 1
    # Tr - 1 ≈ (N-1)e^{-λ₁t} for λ₁ << λ₂ or when λ₁ dominates
    
    # Fit: log(Tr - 1) = log(N-1) - λ₁·t
    y = np.log(np.maximum(traces - 1, 1e-10))
    
    # Linear fit
    n = len(t_values)
    sum_t = sum(t_values)
    sum_y = sum(y)
    sum_ty = sum(ti*yi for ti, yi in zip(t_values, y))
    sum_tt = sum(ti*ti for ti in t_values)
    
    slope = (n * sum_ty - sum_t * sum_y) / (n * sum_tt - sum_t * sum_t)
    intercept = (sum_y - slope * sum_t) / n
    
    lambda1 = -slope  # slope = -λ₁
    
    # R² for fit quality
    y_pred = [slope * t + intercept for t in t_values]
    ss_res = sum((yi - yp)**2 for yi, yp in zip(y, y_pred))
    y_mean = sum_y / n
    ss_tot = sum((yi - y_mean)**2 for yi in y)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    return lambda1, r2, traces

In [ ]:
# Alternative: use eigendecomposition (more accurate)
def extract_lambda1_direct(L):
    """Direct eigenvalue computation for comparison."""
    from scipy.linalg import eigvalsh
    eigs = eigvalsh(L)
    eigs = np.sort(eigs)
    # First non-zero eigenvalue
    lambda1 = eigs[eigs > 1e-8][0] if np.any(eigs > 1e-8) else eigs[1]
    return lambda1

## Test: σ-Independence via Heat Kernel

Si le heat kernel extrait le vrai λ₁ de la variété, différents σ devraient donner le même résultat.

In [ ]:
# Parameters (smaller N due to O(N³) expm cost)
N_VALUES = [500, 800, 1200]
SIGMA_VALUES = [0.4, 0.6, 0.8, 1.0]
T_VALUES = [0.5, 1.0, 2.0, 4.0, 8.0]  # Time points for heat kernel
N_SEEDS = 3

print("Heat Kernel Spectral Estimator")
print("="*60)
print(f"N: {N_VALUES}")
print(f"σ: {SIGMA_VALUES}")
print(f"t: {T_VALUES}")
print(f"Note: Using smaller N due to O(N³) matrix exponential cost")

In [ ]:
%%time
results = []

for N in N_VALUES:
    print(f"\n{'='*50}")
    print(f"N = {N}")
    print(f"{'='*50}")
    
    for seed in range(N_SEEDS):
        theta, q1, q2 = sample_TCS_K7(N, 42 + seed)
        print(f"  Seed {seed}: Computing distances...", end=" ")
        D = compute_distance_matrix(theta, q1, q2)
        print("done")
        
        for sigma in SIGMA_VALUES:
            L = compute_normalized_laplacian(D, sigma)
            
            # Heat kernel method
            lam1_heat, r2, traces = extract_lambda1_from_heat_trace(L, T_VALUES)
            
            # Direct method for comparison
            lam1_direct = extract_lambda1_direct(L)
            
            product_heat = float(lam1_heat * H_STAR)
            product_direct = float(lam1_direct * H_STAR)
            
            results.append({
                'N': N, 'seed': seed, 'sigma': sigma,
                'lambda1_heat': float(lam1_heat),
                'lambda1_direct': float(lam1_direct),
                'product_heat': product_heat,
                'product_direct': product_direct,
                'r2': float(r2)
            })
            
            print(f"    σ={sigma}: Heat={product_heat:.2f}, Direct={product_direct:.2f}, R²={r2:.3f}")
        
        del D, L

print(f"\nCompleted: {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
import matplotlib.pyplot as plt

# Analyze results
print("\n" + "="*60)
print("ANALYSIS: Heat Kernel vs Direct Eigenvalue")
print("="*60)

# Group by N and sigma
for N in N_VALUES:
    print(f"\nN = {N}:")
    print(f"  {'σ':>5} | {'Heat λ₁×H*':>12} | {'Direct λ₁×H*':>12} | {'Match?':>8}")
    print(f"  {'-'*50}")
    
    for sigma in SIGMA_VALUES:
        heat_vals = [r['product_heat'] for r in results if r['N']==N and r['sigma']==sigma]
        direct_vals = [r['product_direct'] for r in results if r['N']==N and r['sigma']==sigma]
        
        heat_mean = np.mean(heat_vals)
        direct_mean = np.mean(direct_vals)
        match = "✓" if abs(heat_mean - direct_mean) < 1 else "✗"
        
        print(f"  {sigma:>5.1f} | {heat_mean:>12.2f} | {direct_mean:>12.2f} | {match:>8}")

In [ ]:
# σ-independence test (using direct values which are more reliable)
print("\n" + "="*60)
print("σ-INDEPENDENCE TEST (Direct Eigenvalue)")
print("="*60)

for N in N_VALUES:
    print(f"\nN = {N}:")
    sigma_means = []
    for sigma in SIGMA_VALUES:
        vals = [r['product_direct'] for r in results if r['N']==N and r['sigma']==sigma]
        mean_val = np.mean(vals)
        sigma_means.append(mean_val)
        print(f"  σ={sigma}: λ₁×H* = {mean_val:.2f}")
    
    spread = max(sigma_means) - min(sigma_means)
    print(f"  Spread: {spread:.2f}")
    if spread < 2:
        print(f"  ✓ Relatively σ-independent")
    else:
        print(f"  ✗ σ-dependent")

In [ ]:
# Visualize heat trace decay
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Example: show trace decay for one configuration
N_example = N_VALUES[-1]
theta, q1, q2 = sample_TCS_K7(N_example, 42)
D = compute_distance_matrix(theta, q1, q2)

ax = axes[0]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(SIGMA_VALUES)))

for i, sigma in enumerate(SIGMA_VALUES):
    L = compute_normalized_laplacian(D, sigma)
    traces = [heat_kernel_trace(L, t) for t in T_VALUES]
    ax.semilogy(T_VALUES, [tr - 1 for tr in traces], 'o-', 
                label=f'σ={sigma}', color=colors[i], markersize=8)

ax.set_xlabel('t', fontsize=12)
ax.set_ylabel('Tr(e^{-tL}) - 1', fontsize=12)
ax.set_title(f'Heat Kernel Trace Decay (N={N_example})', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Right: λ₁×H* vs σ for each N
ax = axes[1]
for i, N in enumerate(N_VALUES):
    means = []
    stds = []
    for sigma in SIGMA_VALUES:
        vals = [r['product_direct'] for r in results if r['N']==N and r['sigma']==sigma]
        means.append(np.mean(vals))
        stds.append(np.std(vals))
    ax.errorbar(SIGMA_VALUES, means, yerr=stds, marker='o', label=f'N={N}',
                capsize=3, linewidth=2, markersize=8)

ax.axhline(y=14, color='red', linestyle='--', alpha=0.7, label='Pell (14)')
ax.axhline(y=13, color='blue', linestyle='--', alpha=0.7, label='Spinor (13)')
ax.set_xlabel('σ', fontsize=12)
ax.set_ylabel('λ₁ × H*', fontsize=12)
ax.set_title('σ-Dependence (Direct Eigenvalue)', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('heat_kernel_analysis.png', dpi=150)
plt.show()

In [ ]:
# Save results
final_results = {
    'metadata': {
        'date': datetime.now().isoformat(),
        'method': 'Heat kernel trace + direct eigenvalue',
        'H_star': H_STAR,
        'N_values': N_VALUES,
        'sigma_values': SIGMA_VALUES,
        't_values': T_VALUES
    },
    'results': results
}

with open('heat_kernel_spectral_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("Saved to heat_kernel_spectral_results.json")

## Conclusion

Le heat kernel trace est une méthode alternative pour extraire λ₁.

**Avantages**:
- Intrinsèque (ne dépend pas de la base)
- Connecté à la géométrie (asymptotics → courbure)

**Limitations**:
- Coût O(N³) pour l'exponentielle matricielle
- Dépend encore du choix de σ pour construire L

**Observation**: Le heat kernel ne résout pas le problème fondamental —
la construction du graphe Laplacien nécessite toujours un paramètre (k, σ, etc.).